# ジェネレータ ── メモリ効率のよいイテラブルなオブジェクト

## ジェネレータの具体例

In [ ]:
# yieldを含む関数はジェネレータになる
def inf(n):
    while True:
        yield n

## ジェネレータの実装

### ジェネレータ関数 ── 関数のように作成する

In [ ]:
def gen_function(n):
    print('start')
    while n:
        print(f'yield: {n}')
        yield n  # ここで一時中断される
        n -= 1

In [3]:
# 戻り値はジェネレータイテレータ
gen = gen_function(2)
gen

<generator object gen_function at 0x109ef6970>

In [4]:
# 組み込み関数next()に渡すと
# __next__()が呼ばれる
next(gen)

start
yield: 2


2

In [5]:
next(gen)

yield: 1


1

In [6]:
next(gen)

StopIteration: 

In [7]:
def gen_function(n):
    while n:
        yield n
        n -= 1

In [8]:
# for文での利用
for i in gen_function(2):
    print(i)

2
1


In [9]:
# 内包表記での利用
[i for i in gen_function(5)]

[5, 4, 3, 2, 1]

In [10]:
# イテラブルを受け取る関数に渡す
max(gen_function(5))

5

### ジェネレータ式 ── 内包表記を利用して作成する

In [11]:
x = [1, 2, 3, 4, 5]

In [12]:
# これはリスト内包表記
listcomp = [i**2 for i in x]
listcomp  # すべての要素がメモリ上にすぐ展開される

[1, 4, 9, 16, 25]

In [13]:
# これはジェネレータ式
gen = (i**2 for i in x)
gen  # 各要素は必要になるまで計算されない

<generator object <genexpr> at 0x10a839350>

In [14]:
# リストにすると最後の要素まで計算される
list(gen)

[1, 4, 9, 16, 25]

In [15]:
x = [1, 2, 3, 4, 5]

# max((i**3 for i in x))と等価
max(i**3 for i in x)

125

### yield from式 ── サブジェネレータへ処理を委譲する

In [16]:
def chain(iterables):
    for iterable in iterables:
        for v in iterable:
            yield v

In [17]:
iterables = ('python', 'book')

In [18]:
list(chain(iterables))

['p', 'y', 't', 'h', 'o', 'n', 'b', 'o', 'o', 'k']

In [19]:
def chain(iterables):
    for iterable in iterables:
        yield from (v for v in iterable)

In [20]:
list(chain(iterables))

['p', 'y', 't', 'h', 'o', 'n', 'b', 'o', 'o', 'k']

## ジェネレータを利用する際の注意点

In [21]:
def gen(n):
    while n:
        yield n
        n -= 1

In [22]:
# zip()にリストとジェネレータを同時に渡す
x = [1, 2, 3, 4, 5]
[i for i in zip(x, gen(5))]

[(1, 5), (2, 4), (3, 3), (4, 2), (5, 1)]

In [23]:
# filter()にジェネレータを渡す
odd = filter(lambda v: v % 2 == 1, gen(5))
[i for i in odd]

[5, 3, 1]

### len()で利用する場合

In [24]:
len(gen(5))

TypeError: object of type 'generator' has no len()

In [25]:
len(list(gen(5)))

5

In [26]:
# 値を無限に返すジェネレータ
g = gen(-1)

### 複数回利用する場合

In [27]:
g = gen(4)
len(list(g))

4

In [28]:
len(list(g))

0

In [29]:
g = gen(4)
list_nums = list(g)
len(list_nums)

4

In [30]:
len(list_nums)

4

## ジェネレータの実例 ── ファイルの内容を変換する

In [31]:
def convert(line):
    return line.upper()

In [32]:
def reader(src):
    with open(src) as f:
        for line in f:
            yield line

In [33]:
def writer(dest, reader):
    with open(dest, 'w') as f:
        for line in reader:
            f.write(convert(line))

## そのほかのユースケース

# デコレータ ── 関数やクラスの前後に処理を追加する

## デコレータの具体例

### functools.lru\_cache() ── 関数の結果をキャッシュする関数デコレータ

In [34]:
from functools import lru_cache
from time import sleep

# 最近の呼び出し最大32回分までキャッシュ
@lru_cache(maxsize=32)
def heavy_funcion(n):
    sleep(3)  # 重い処理をシミュレート
    return n + 1

In [35]:
# 初回は時間がかかる
heavy_funcion(2)

3

In [36]:
# キャッシュにヒットするのですぐに結果を得られる
heavy_funcion(2)

3

### dataclasses.dataclass() ── よくある処理を自動追加するクラスデコレータ

In [37]:
from dataclasses import dataclass
@dataclass(frozen=True)
class Fruit:
    name: str  # 型ヒントを付けて属性を定義
    price: int = 0  # 初期値も指定

In [38]:
# __init__()や__repr__()が追加されている
apple = Fruit(name='apple', price=128)
apple

Fruit(name='apple', price=128)

In [39]:
# frozen=Trueとしたので読み取り専用
apple.price = 256

FrozenInstanceError: cannot assign to field 'price'

## デコレータの実装

### シンプルなデコレータ

In [40]:
# デコレートしたい関数を受け取る
def deco1(f):
    print('deco1 called')
    def wrapper():
        print('before exec')
        v = f()  # もとの関数を実行
        print('after exec')
        return v
    return wrapper

In [41]:
# デコレータは関数定義時に実行される
@deco1
def func():
    print('exec')
    return 1

deco1 called


In [42]:
# すでにdeco1(func)の結果へ置き換わっている
func.__name__

'wrapper'

In [43]:
func()  # wrapper()と等価

before exec
exec
after exec


1

In [44]:
@deco1
def func(x, y):
    print('exec')
    return x, y

deco1 called


In [45]:
# deco1内のv = f()の行でエラー
func(1, 2)

TypeError: wrapper() takes 0 positional arguments but 2 were given

### 引数を受け取る関数のデコレータ

In [46]:
def deco2(f):
    # 新しい関数が引数を受け取る
    def wrapper(*args, **kwargs):
        print('before exec')
        # 引数を渡してもとの関数を実行
        v = f(*args, **kwargs)
        print('after exec')
        return v
    return wrapper

In [47]:
@deco2
def func(x, y):
    print('exec')
    return x, y

In [48]:
func(1, 2)

before exec
exec
after exec


(1, 2)

### デコレータ自身が引数を受け取るデコレータ

In [49]:
# 引数zを受け取る
def deco3(z):
    # deco2()と同等
    def _deco3(f):
        def wrapper(*args, **kwargs):
            # ここでzを参照できる
            print('before exec', z)
            v = f(*args, **kwargs)
            print('after exec', z)
            return v
        return wrapper
    return _deco3  # デコレータを返す

In [50]:
# deco3(z=3)の戻り値がデコレータの実体
# つまりfunc = deco3(z=3)(func)と同等
@deco3(z=3)
def func(x, y):
    print('exec')
    return x, y

In [51]:
# zに渡した値は保持されている
func(1, 2)

before exec 3
exec
after exec 3


(1, 2)

### 複数のデコレータを同時に利用する

In [52]:
# 複数のデコレータを利用
@deco3(z=3)
@deco3(z=4)
def func(x, y):
    print('exec')
    return x, y

In [53]:
# @deco3(z=4)が適用された結果に
# @deco3(z=3)が適用される
func(1, 2)

before exec 3
before exec 4
exec
after exec 4
after exec 3


(1, 2)

### functools.wraps()でデコレータの欠点を解消する

In [54]:
from functools import wraps
def deco4(f):
    @wraps(f)  # もとの関数を引数に取るデコレータ
    def wrapper(*args, **kwargs):
        print('before exec')
        v = f(*args, **kwargs)
        print('after exec')
        return v
    return wrapper

In [55]:
@deco4
def func():
    """funcです"""
    print('exec')

In [56]:
func.__name__

'func'

In [57]:
func.__doc__

'funcです'

## デコレータの実例 ── 処理時間の計測

In [58]:
from functools import wraps
import time
def elapsed_time(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        start = time.time()
        v = f(*args, **kwargs)
        print(f"{f.__name__}: {time.time() - start}")
        return v
    return wrapper

In [59]:
# 0からn-1までの総和を計算する関数
@elapsed_time
def func(n):
    return sum(i for i in range(n))

In [60]:
# func()の実行結果を表示
# f-stringで数値のカンマ（,）区切りを指定
print(f'{func(1000000)=:,}')

func: 0.2556889057159424
func(1000000)=499,999,500,000


In [61]:
print(f'{func(10000000)=:,}')

func: 0.65425705909729
func(10000000)=49,999,995,000,000


# コンテキストマネージャー ── with文の前後で処理を実行するオブジェクト

## コンテキストマネージャーの具体例

In [62]:
# 第二引数で書き込みモードを指定
with open('some.txt', 'w') as f:
    f.write('python')

In [63]:
f.closed

True

In [64]:
with open('some.txt', 'w') as f:
    f.read()  # 書き込みモードなので例外になる

UnsupportedOperation: not readable

In [65]:
# 例外発生時もクローズされている
f.closed

True

In [66]:
try:
    f = open('some.txt', 'w')
    f.read()  # 書き込みモードなので例外になる
finally:
    f.close()

UnsupportedOperation: not readable

In [67]:
# 例外発生時もクローズされた
f.closed

True

## コンテキストマネージャーの実装

### \_\_enter\_\_()、\_\_exit\_\_() ── with文の前後に呼ばれるメソッド

In [68]:
# このクラスのインスタンスがコンテキストマネージャー
class ContextManager:
    # 前処理を実装
    def __enter__(self):
        print('__enter__ was called')
    # 後処理を実装
    def __exit__(self, exc_type, exc_value, traceback):
        print('__exit__ was called')
        print(f'{exc_type=}')
        print(f'{exc_value=}')
        print(f'{traceback=}')

In [69]:
# withブロックが正常終了の場合は
# __exit__()の引数はすべてNone
with ContextManager():
    print('inside the block')

__enter__ was called
inside the block
__exit__ was called
exc_type=None
exc_value=None
traceback=None


### with文と例外処理

In [70]:
# withブロック内で例外が発生した場合は
# その情報が__exit__()に渡される
with ContextManager():
    1 / 0

__enter__ was called
__exit__ was called
exc_type=<class 'ZeroDivisionError'>
exc_value=ZeroDivisionError('division by zero')
traceback=<traceback object at 0x10a8954c0>


ZeroDivisionError: division by zero

### asキーワード ── \_\_enter\_\_()の戻り値を利用する

In [71]:
class ContextManager:
    # 戻り値がasキーワードに渡される
    def __enter__(self):
        return 1
    def __exit__(self, exc_type, exc_value, traceback):
        pass

In [72]:
with ContextManager() as f:
    print(f)

1


In [73]:
# asキーワードの省略
with ContextManager():
    pass

In [74]:
class Point:
    def __init__(self, **kwargs):
        self.value = kwargs
    def __enter__(self):
        print('__enter__ was called')
        return self.value  # as節で渡される
    def __exit__(self, exc_type, exc_value, traceback):
        print('__exit__ was called')
        print(self.value)

In [75]:
with Point(x=1, y=2) as p:
    print(p)
    p['z'] = 3

__enter__ was called
{'x': 1, 'y': 2}
__exit__ was called
{'x': 1, 'y': 2, 'z': 3}


### contextlib.contextmanagerでシンプルに実装する

In [76]:
from contextlib import contextmanager
@contextmanager
def point(**kwargs):
    print('__enter__ was called')
    value = kwargs
    try:
        # ここより上が前処理
        # valueをasキーワードに渡す
        yield value
        # ここより下が後処理
    except Exception as e:
        # エラー時はこちらも呼ばれる
        print(e)
        raise
    finally:
        print('__exit__ was called')
        print(value)

In [77]:
with point(x=1, y=2) as p:
    print(p)
    p['z'] = 3

__enter__ was called
{'x': 1, 'y': 2}
__exit__ was called
{'x': 1, 'y': 2, 'z': 3}


## コンテキストマネージャーの実例 ── 一時的なログレベルの変更

In [78]:
!cat debug_context.py

import logging
from contextlib import contextmanager

logger = logging.getLogger(__name__)
logger.addHandler(logging.StreamHandler())

# デフォルトをINFOレベルとし、DEBUGレベルのログは無視する
logger.setLevel(logging.INFO)

@contextmanager
def debug_context():
    level = logger.level
    try:
        # ログレベルを変更する
        logger.setLevel(logging.DEBUG)
        yield
    finally:
        # もとのログレベルに戻す
        logger.setLevel(level)

def main():
    logger.info('before: info log')
    logger.debug('before: debug log')

    # DEBUGログを見たい処理をwithブロック内で実行する
    with debug_context():
        logger.info('inside the block: info log')
        logger.debug('inside the block: debug log')

    logger.info('after: info log')
    logger.debug('after: debug log')

if __name__ == '__main__':
    main()

In [79]:
!python3 debug_context.py

before: info log
inside the block: info log
inside the block: debug log
after: info log


## そのほかのユースケース

# デスクリプタ ── 属性処理をクラスに委譲する

## デスクリプタの具体例

In [80]:
# デスクリプタが持つメソッドが定義されている
dir(property())

['__class__',
 '__delattr__',
 '__delete__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__isabstractmethod__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__set__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'deleter',
 'fdel',
 'fget',
 'fset',
 'getter',
 'setter']

In [81]:
# propertyの実体はクラスとして定義されている
type(property())

property

In [82]:
class A:
    def f(self):
        pass

In [83]:
# デスクリプタが持つメソッドが定義されている
dir(A.f)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [84]:
type(A.f)

function

## デスクリプタの実装

### \_\_set\_\_()を実装する ── データデスクリプタ

In [85]:
# __set__()を持つクラスはデータデスクリプタ
class TextField:
    def __set_name__(self, owner, name):
        print(f'__set_name__ was called')
        print(f'{owner=}, {name=}')
        self.name = name
    def __set__(self, instance, value):
        print('__set__ was called')
        if not isinstance(value, str):
            raise AttributeError('must be str')
        # ドット記法ではなく属性辞書を使って格納
        instance.__dict__[self.name] = value
    def __get__(self, instance, owner):
        print('__get__ was called')
        return instance.__dict__[self.name]

In [86]:
class Book:
    title = TextField()

__set_name__ was called
owner=<class '__main__.Book'>, name='title'


In [87]:
book = Book()

In [88]:
# 代入時には__set__()が呼ばれる
book.title = 'Python Practice Book'

__set__ was called


In [89]:
# 取得時には__get__()が呼ばれる
book.title

__get__ was called


'Python Practice Book'

In [90]:
# 別のインスタンスを作成して代入
notebook = Book()

In [91]:
notebook.title = 'Notebook'

__set__ was called


In [92]:
# それぞれデータを保持している
book.title

__get__ was called


'Python Practice Book'

In [93]:
notebook.title

__get__ was called


'Notebook'

In [94]:
# 文字列以外は代入できない
book.title = 123

__set__ was called


AttributeError: must be str

### \_\_get\_\_()のみを実装する ── 非データデスクリプタ

In [95]:
# __get__()のみであれば非データデスクリプタ
class TextField:
    def __init__(self, value):
        if not isinstance(value, str):
            raise AttributeError('must be str')
        self.value = value
    def __set_name__(self, owner, name):
        print(f'__set_name__ was called')
        print(f'{owner=}, {name=}')
        self.name = name
    def __get__(self, instance, owner):
        print('__get__ was called')
        return self.value

In [96]:
class Book:
    title = TextField('Python Practice Book')

__set_name__ was called
owner=<class '__main__.Book'>, name='title'


In [97]:
book = Book()

In [98]:
# 代入前の取得時には__get__()が呼ばれる
book.title

__get__ was called


'Python Practice Book'

In [99]:
# 代入するとインスタンス変数になる
book.title = 'Book'

In [100]:
# インスタンス変数があると__get__()は呼ばれない
book.title

'Book'

## デスクリプタの実例 ── プロパティのキャッシュ

In [101]:
class LazyProperty:
    def __init__(self, func):
        self.func = func
        self.name = func.__name__
    def __get__(self, instance, owner):
        if not instance:
            # クラス変数としてアクセスされたときの処理
            return self
        # self.funcは関数なので明示的にインスタンスを渡す
        v = self.func(instance)
        instance.__dict__[self.name] = v
        return v

In [102]:
TAX_RATE = 1.10

In [103]:
class Book:
    def __init__(self, raw_price):
        self.raw_price = raw_price
    @LazyProperty
    def price(self):
        print('calculate the price')
        return int(self.raw_price * TAX_RATE)

In [104]:
book = Book(1980)
book.price

calculate the price


2178

In [105]:
book.price

2178

In [106]:
Book.price

## そのほかのユースケース

# 本章のまとめ